In [2]:
import numpy as np
import random as rnd
import scipy.spatial.distance as distance
import math
import scipy.stats as stats
from sklearn.preprocessing import normalize

In [5]:
D = 4; K = 0; Beta = 0.1; delta = 0.01
mus = np.ndarray(D + 1); sigmas = np.ndarray(D+1)
for i in range(D + 1):
    mus[i] = 2 * i / 3 * D
    sigmas[i] = (i % 3 + 2) / D

startingMus = mus
numvectors = 10; lenvectors = 10
X = np.ndarray((numvectors, lenvectors))

for i in range(numvectors):
    index = i % (D + 1)
    
    summed = 0
    for j in range(lenvectors):
        X[i,j] = rnd.gauss(mus[index], sigmas[index])
        summed += X[i,j] ** 2
    #print(X[i])
    #print(np.linalg.norm(X[i]))
    
    for j in range(lenvectors):
        X[i,j] = X[i,j] / math.sqrt(summed)
    
    #print(X[i])
    #print(np.linalg.norm(X[i]))
    #print()


In [6]:
sigma = np.std(X)
sigmaInv = sigma ** -1
#print(sigmaInv)
alpha = sigmaInv * np.identity(lenvectors)
covarDet = np.linalg.det(alpha) ** -1
#print(alpha)

alphas = {}
mus = {}
covDets = {}
comProbs = {}
ages = {}
accumulators ={}

alphas[0] = alpha
covDets[0] = covarDet
mus[0] = X[0]
comProbs[0] = 1
ages[0] = 1
accumulators[0] = 1
K = 1

In [7]:
for i in range(len(X)):
    if i == 0:
        continue
    updated = False
    
    for j in range(K):
        if i == j:
            continue
        
        print('(' + str(i) + ', ' + str(j) + ')')

        dist = distance.mahalanobis(X[i], mus[j], alphas[j]) ** 2
        print(dist)
        #dist2 = stats.multivariate_normal.logpdf(X[i], mus[j], np.linalg.inv(alpha))
        #dist3 = math.exp(dist2)
        #print(dist2)
        #print(dist3)
        dist2 = stats.multivariate_normal.pdf(X[i], mus[j], np.linalg.inv(alpha))
        print(dist2)
        
        if dist < dist2:
            print('call update')
            print()
            update(accumulators, ages, alphas, comProbs, covDets, D, j, K, mus, sigmaInv, X[i])
            updated = True
            
    if not updated:
        print('call create')
        print()
        K = create(accumulators, ages, alphas, comProbs, covDets, K, mus, sigmaInv, X[i])
    

(1, 0)
12.906805020494039
0.0003494207195308792
call create

(2, 0)
12.852237253061576
0.0003590855209286274
(2, 1)
0.37254087705883965
0.18413210997901452
call create

(3, 0)
12.578726602519062
0.00041170863610357834
(3, 1)
0.325185268606775
0.18854397960340305
(3, 2)
0.17524925411436046
0.20322206562170897
call update

(4, 0)
12.574127794961038
0.00041265640973682933
(4, 1)
0.36624113768250655
0.18471301654054317
(4, 2)
0.19197921567377543
0.20321886535348022
call update

(5, 0)
6.449824334060999
0.00881991083443832
(5, 1)
12.305755085918062
0.00047191631141797963
(5, 2)
878.7246226156566
0.0005661516027517795
call create

(6, 0)
11.135937156785959
0.0008470084136833295
(6, 1)
0.9555545421656132
0.13757189544815318
(6, 2)
0.9573966148168946
0.18404214401977434
(6, 3)
11.145296049523962
0.0008430541423533405
call create

(7, 0)
12.17636666546834
0.0005034557753146552
(7, 1)
0.432261358757976
0.17871515904548851
(7, 2)
0.4299939960425805
0.19103589606796023
(7, 3)
11.186468810443241
0.

In [3]:
def create(accumulators, ages, alphas, comProbs, covDets, K, mus, sigmaInv, x):
    accumulators[K] = 1
    ages[K] = 1
    alphas[K] = sigmaInv * np.identity(len(alphas[0]))
    comProbs[K] = 1 / sum(accumulators.values())
    covDets[K] = np.linalg.det(alphas[K]) ** -1
    mus[K] = x
    K += 1
    return K


In [4]:
def update(accumulators, ages, alphas, comProbs, covDets, D, j, K, mus, sigmaInv, x):
    dist = distance.mahalanobis(x, mus[j], alphas[j]) ** 2
    pxj = 1 / ((2 * math.pi) ** (D / 2) * math.sqrt(covDets[j])) * math.exp(-0.5 * dist)
    
    pjx = pxj * comProbs[j]
    totProbs = 0
    for i in range(K):
        pxi =  1 / ((2 * math.pi) ** (D / 2) * math.sqrt(covDets[i])) * math.exp(-0.5 * dist)
        totProbs += pxi * comProbs[i]
    pjx /= totProbs
    
    ages[j] += 1
    accumulators[j] += pjx
    
    ej = x - mus[j]
    weight = pjx / accumulators[j]
    
    deltaMu = weight * ej
    mus[j] += deltaMu
    
    oldAlpha = alphas[j]
    ejStar = x - mus[j]
    newAlpha = oldAlpha / (1 - weight) - (weight / (1 - weight) ** 2) * np.matmul(np.matmul(oldAlpha, ejStar), np.matmul(ejStar.transpose(), oldAlpha)) \
            / (1 + (weight / (1 - weight)) * np.matmul(np.matmul(ejStar.transpose(), oldAlpha),ejStar))
    alphas[j] = newAlpha +  (np.matmul(np.matmul(newAlpha, mus[j]), np.matmul(mus[j].transpose(), newAlpha))) \
            / (1 - np.matmul(np.matmul(mus[j].transpose(), newAlpha), mus[j]))
    
    comProbs[j] = accumulators[j] / sum(accumulators.values())
    
    newCovDet = (1 - weight) ** D * covDets[j] * (1 + weight/(1 - weight) * np.matmul(ejStar.transpose(), np.matmul(oldAlpha, ejStar)))
    covDets[j] = newCovDet * (1 - np.matmul(deltaMu.transpose(), np.matmul(newAlpha, deltaMu)))
    
    

In [8]:
print(K)
print(mus)
print(startingMus)

6
{0: array([-0.2727993 ,  0.31849613, -0.39834572,  0.03773479, -0.2803894 ,
       -0.08332718, -0.16847103,  0.49117129, -0.54791894, -0.09289244]), 1: array([0.22575483, 0.31766415, 0.43468354, 0.35819787, 0.34384541,
       0.3270145 , 0.37478524, 0.13019586, 0.21020194, 0.32264656]), 2: array([0.2653357 , 0.25613134, 0.360573  , 0.33831841, 0.33682299,
       0.3712395 , 0.23872625, 0.27039938, 0.34257823, 0.3335331 ]), 3: array([ 0.43886984,  0.28299853, -0.40302317,  0.18890139, -0.65723543,
       -0.1673771 , -0.18220265, -0.11602215,  0.0315711 , -0.14684859]), 4: array([0.19635228, 0.29056921, 0.17212216, 0.37462874, 0.25536963,
       0.4885874 , 0.25226761, 0.38296071, 0.34773229, 0.26813371]), 5: array([0.35301797, 0.4058724 , 0.32008208, 0.21361708, 0.31399312,
       0.33443066, 0.29447418, 0.26063729, 0.27892975, 0.3409187 ])}
[ 0.          2.66666667  5.33333333  8.         10.66666667]
